# Value reassignment in numpy -- perfomance comparison between numba loops and numpy LUT


### Problem definition

We want to address a simple problem. Given a numpy array **A** containing integer values, we want to create a numpy array **B**, in which all elements in **A** are replaced by a value map that maps integer values that are present in **A** to some target values (not necessarily integers).

### Representing the mapping

The mapping can be represented in several ways:

1. Two vectors of the same length. One vector (`input_vals`) contains the source values and the other vector (`dest_vals`) contains the destination values. For each pixel in **A**, one needs to find the index `i` at which `input_vals[i] == A[pixel]`. The replacement value is then found at the same index position in `dest_vals`, i.e. we can assign `B[pixel]=dest_val[i]`. The two vectors can also be represented as a table with two columns or and array with two columns if the input and output data types are identical. 
2. A lookup table **L** (represented as vector/array). For a given pixel value `A[pixel]` the pixel value is used as the index to the lookup table `B[pixel]=L[A[pixel]]`
3. A hash table with lookup values. The idea is the same as in 2. but the representation is different(for example a `dict`).

The different approaches each have their pros and cons:

1. When using two vectors, we need to search the `input_vals` at each pixel to find the output pixel. So the complexity is `O(n_pixels) * O(n_different_input_values)`. We can implement a few speed-ups if we make assumptions about the input data, but the asymptotic complexity remains the same. The memory footprint depends only on the size of the input value and the number of different input values in the source image. This approach (looping over pixels) is not very fast to apply in numpy, therefore we will use numba in the examples below to speed up the loop.
2. For the lookup table approach the complexity is always `O(n_pixels)` as the lookup takes constant time. However, the size of the lookup table depends on the maximum value of the lookup table, not on the number of different values. For many practical problems, that is not an issue but conceptually that is not very nice. This implementation can be implemented easily with numpy arrays, which make use of vectorization and are thus very fast.
3. Implementing a LUT as a hash table (sparse array) does have the same complexity `O(n_pixels)` as the lookup is still happening in constant time. The LUT memory requirements for a hash table only scale with the number of different values in **A**. So in theory, this should be the best option. However, numpy does not implement sparse arrays, so we have to rely on python dictionaries for hashing, which is much less efficiently implemented.

In [1]:
import numpy as np
import numba
import matplotlib.pyplot as plt
import warnings
import skimage.segmentation
numba.__version__

'0.48.0'

In [2]:
@numba.jit(nogil=True, nopython=True)
def _reassign(inarr, resarr, invals, outvals):
    for i in range(len(inarr)):
        for j in range(len(invals)):
            if inarr[i] == invals[j]:
                resarr[i] = outvals[j]

Let's implement a simple reassignment loop in pure python:

In [3]:
def _reassign_pure_python(inarr, resarr, invals, outvals):
    """ simple reassignment, making no assumptions about the data"""
    for i in range(len(inarr)):
        for j in range(len(invals)):
            if inarr[i] == invals[j]:
                resarr[i] = outvals[j]

Implementing this with numba is a simple matter of adding a decorator

In [4]:
@numba.jit(nogil=True, nopython=True)
def _reassign_simple(inarr, resarr, invals, outvals):
    """ simple reassignment, making no assumptions about the data"""
    for i in range(len(inarr)):
        for j in range(len(invals)):
            if inarr[i] == invals[j]:
                resarr[i] = outvals[j]

Every pixel can be processed independently, so we give numba some hints about
parralelization by using `numba.prange` instead of `range` and by setting `parallel=True` in the decorator.

In [5]:
@numba.jit(nogil=True, nopython=True, parallel=True)
def _reassign_parallel(inarr, resarr, invals, outvals):
    """ simple reassignment, with some hints for paralllelization"""
    for i in numba.prange(len(inarr)):
        for j in range(len(invals)):
            if inarr[i] == invals[j]:
                resarr[i] = outvals[j]

We can add a few optimizations if we make assumptions about the type of data we process.
In image analysis, one often comes across so-called label images, where each integer value represents an object. Often, most of the image area will be background, with some connected components as foreground. We add some simple mechanism to speed up the lookup for the background pixel and to cache the most recent lookup.
It also makes sense to pass in the vectors for input and output values sorted by the frequency of the values if available (most frequent values first).

In [6]:
@numba.jit(nogil=True, nopython=True)    
def _reassign_optimized(inarr, resarr, invals, outvals):
    """ optimized reassignment, making the following assumptions
    about the data:
    1. zeroth element of the invals is the most frequent value (background
    label)
    2. array contains connected components which often results in
    neighbouring pixels having the same value. 
    """
    zeroval = invals[0]
    zeroval_replace = outvals[0]
    lastval = zeroval
    lastval_replace = zeroval_replace
    for i in range(len(inarr)):
        if inarr[i] == lastval:
            resarr[i] == lastval_replace
        elif inarr[i] == zeroval:
            resarr[i] == zeroval_replace
        else: 
            for j in range(len(invals)):
                if inarr[i] == invals[j]:
                    resarr[i] = outvals[j]
                    lastval = invals[j]
                    lastval_replace = outvals[j]

In [20]:
def _reassign_dict(inarr, resarr, invals, outvals):
    @numba.jit(nogil=True, nopython=True, parallel=True)
    def _reassign_dict_inner(inarr, resarr, lut_dict):
        """ simple reassignment, with some hints for paralllelization"""
        for i in numba.prange(len(inarr)):
            resarr[i] = lut_dict[inarr[i]]
    # turn the inval/outval vectors into a dict
    lut = dict(zip(list(invals),list(outvals)))
    _reassign_dict_inner(inarr, resarr, lut)

In [57]:
# TODO: currently this is hardcoded to int64 -> int64
# need to check out how to automatically
# create the correct numba types for the numba.typed.Dict
# from numpy.dtypes
from numba.typed import Dict
from numba import types
@numba.jit(nogil=True, nopython=True, parallel=True)
def _reassign_dict(inarr, resarr, invals, outvals):
    lut = Dict.empty(
        key_type=types.int64,
        value_type=types.int64)
    for key, value in zip(invals,outvals):
        lut[key]=value
    for i in numba.prange(len(inarr)):
        resarr[i] = lut[inarr[i]]

Let's add a wrapper function that checks the input values and types, gives a flattened view of the input array and allocates space for an output array:

In [10]:
def reassign(input_arr, input_vals, output_vals, _reassign, inplace=False ):
    """
    input_arr: input array (np.ndarray)
    input_vals: 1d array of input values (integer)
    output_vals: 1d array of output values
    _reassign: function that does the actual reassigment
    inplace: whether the reassignment should happen inplace
    """

    assert issubclass(input_arr.dtype.type, np.integer), "Reassignment only possible for integer types"
    assert issubclass(input_vals.dtype.type, np.integer), "Reassignment only possible for integer types"
    assert input_vals.ndim == 1, "input_vals must be a 1D array"
    assert input_vals.shape == output_vals.shape, "output_vals must have the same shape as input_vals"

    # We want to reshape to 1D to make the numba loop as simple
    # as possible
    orig_shape = input_arr.shape
    # numpy doc for ravel says 
    # "When a view is desired in as many cases as possible, 
    # arr.reshape(-1) may be preferable."
    input_arr = input_arr.reshape(-1) 
    
    if inplace:
        if input_vals.dtype == output_vals.dtype:
            out_arr = input_arr
        else:
            raise TypeError("input_vals and output_vals dtypes must match for inplace operation")
    else:
        out_arr = np.empty_like(input_arr, dtype=output_vals.dtype)

    _reassign(input_arr, out_arr, input_vals, output_vals)
    return out_arr.reshape(orig_shape)

Now, lets implement something that does essentially the same as `skimage.segmentation.relabel_sequential`. This is a common operation on label images, e.g. after some filtering step that removes some labels. 

In [11]:
def relabel_sequential(label, _reassign, inplace=False, sort=False):
    unique_vals, counts = np.unique(label, return_counts=True)
    sequential_vals = np.empty_like(unique_vals, label.dtype)
    sequential_vals[:] = np.arange(len(unique_vals))
    # sort such that largest counts are first for the value 
    # lookup. This is an optimization for reassign.
    if sort:
        idx = np.argsort(counts)[::-1] 
        sequential_vals = sequential_vals[idx].copy()
        unique_vals = unique_vals[idx].copy()
    relabeled = reassign(label, unique_vals, sequential_vals, _reassign, inplace=inplace)
    return relabeled

Create some test data. Note that such random data is the worst case, where none of the assumptions for optimization will hold (no connected components, background is not the most common pixel type)

In [49]:
labelarr = np.random.randint(0,10000, size=(1000,1000))


Time the different implementations:

In [50]:
# use skimage.segmentation.relabel_sequential as baseline
# that uses numpy arrays as LUTs
%timeit relabeled = skimage.segmentation.relabel_sequential(labelarr)[0]

86.7 ms ± 3.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
# simple loop with numba jit 
%timeit relabeled = relabel_sequential(labelarr, _reassign_simple)

6.61 s ± 77.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
# simple loop with numba jit and parallelization 
%timeit relabeled = relabel_sequential(labelarr, _reassign_parallel)

1.5 s ± 92.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
# simple loop with optimizations for connected components
# (not expected to perform better on random data)
%timeit relabeled = relabel_sequential(labelarr, _reassign_optimized, sort=True)

12.9 s ± 230 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
# simple loop with LUT implemented as dictionary
%timeit relabeled = relabel_sequential(labelarr, _reassign_dict)

144 ms ± 3.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# finally the pure python implementation
# I interrupted this after several minutes (!!!) as I couldn't bear waiting
%timeit relabeled = relabel_sequential(labelarr, _reassign_pure_python)

## Preliminary conclusion:

* numpy LUTs as in `skimage.segmentation.relabel_sequential` perform fastest,
* for a large number of objects and random label locations (no connected components), the simple numba loop is significantly slower than numpy LUTs,
* numba loop with hash table LUT (dictionary) performance is similar order of magnitude as numpy LUTs (mostly a factor of two, pretty independent of input array size and number of labels ... variation of sizes not shown here)

## TODO

* automatically create the correct numba.typed.Dict for different input/output dtypes
* test optimized implementation on actual connected component images (just out of curiosity, hash table is really the preferred method as the lookup always takes constant time)
